# Modifying data

## Apply structures

### apply vs. for

There is a popular belief that for-loops are slow in R. This is not true, but using them improperly can lead to complicated code with a risk of a bad implementation. `apply`-functions are usually recommended as they fit better into R's procedural programming paradigm.

As an example lets try get a sum each row of a random `data.frame` with four columns:

In [15]:
nrows = 100
data <- data.frame(
    var1=rnorm(nrows),
    var2=rnorm(nrows),
    var3=rnorm(nrows),
    var4=rnorm(nrows)
    )

Lets create a simple function `rowSums_forloop` that initializes a result vector `rowsum` and in a for-loop goes through the rows of the input `data.frame` and stores the sum of row `i` to the `rowsum[i]`:

In [55]:
library(pryr)

rowSums_forloop_rows <- function (dataframe) {
    # Sums up each row in a for-loop with row-major order
    # 
    # Args:
    #   dataframe: data.frame to sum over
    #
    # Returns:
    #   rowsum: Vector containing sum of each row
    
    # Generate the vector rowsum and initalize its size
    rowsum <- vector('double',nrow(dataframe))
    
    # Not like this:
    # Generate the vector rowsum, but do not initialize its size
    # rowsum <- c()
    for (i in 1:nrow(dataframe)) {
        for (j in 1:ncol(dataframe)) {

        # Check memory address of rowsum to verify that it does not change
        # print(address(rowsum))
        
        # For each row and column add the value to rowsum[i]
        rowsum[i] <- rowsum[i] + dataframe[i,j]
        }
    }
    return(rowsum)
}

rowSums_forloop_cols <- function (dataframe) {
    # Sums up each row in a for-loop with column-major order
    # 
    # Args:
    #   dataframe: data.frame to sum over
    #
    # Returns:
    #   rowsum: Vector containing sum of each row
    
    # Generate the vector rowsum and initalize its size
    rowsum <- vector('double',nrow(dataframe))
    
    for (j in 1:ncol(dataframe)) {
        for (i in 1:nrow(dataframe)) {
        
            # For each column and row add the value to rowsum
            rowsum[i] <- rowsum[i] + dataframe[i,j]
        }
    }
    return(rowsum)
}

rowSums_forloop_colnames <- function (dataframe) {
    # Sums up each row with one column at the time in a for-loop
    # 
    # Args:
    #   dataframe: data.frame to sum over
    #
    # Returns:
    #   rowsum: Vector containing sum of each row
    
    # Generate the vector rowsum and initalize its size
    rowsum <- vector('double',nrow(dataframe))
    
    for (col in colnames(dataframe)) {
        # For each column add the values to rowsum
        rowsum <- rowsum + dataframe[,col]
    }
    return(rowsum)
}


rowsum_forloop_rows <- rowSums_forloop_rows(data)
rowsum_forloop_cols <- rowSums_forloop_cols(data)
rowsum_forloop_colnames <- rowSums_forloop_colnames(data)

head(rowsum_forloop_rows)
# Check that results match
all.equal(rowsum_forloop_rows,rowsum_forloop_cols)
all.equal(rowsum_forloop_rows,rowsum_forloop_colnames)

[1]  5.2983201 -0.3383926  3.3741513 -0.9845167 -0.9465403 -3.5574545

[1] TRUE

[1] TRUE

One of the previous examples utilizes R's vectorized nature properly while others are a really bad implementation. Can you spot which one is the good one?

Vectorized in this context means that if were going to use the same function `FUN` on each row (column) one at a time, we can just initialize the function once and run the data through it one row (column) at a time. Time is saved as `FUN` can re-used for each iteration of the loop. 

To utilize the vectorized nature of R objects one usually uses `apply`-style functions.

The basic `apply`-function has a simple syntax (see: [[1]](https://www.rdocumentation.org/packages/base/versions/3.4.3/topics/apply)):

```r
apply(X, MARGIN, FUN)
```

Here `X` is a data array (vector, matrix, data.frame); `MARGIN` is a integer/vector that determines whether to apply the function on rows (`1`), columns(`2`) or both (`c(1,2)`); and `FUN` is the function to apply.

Dimensions of the result depend on the dimensions of the original data, the direction of application and the output shape of `FUN`. In this case we want to apply the `sum` function to each row. `sum` returns a single number as a result, so the result size is a vector with `nrows`.

In [53]:
rowSums_apply <- function(dataframe) {
    # Sums up each row with an apply-function call
    # 
    # Args:
    #   dataframe: data.frame to sum over
    #
    # Returns:
    #   rowsum: Vector containing sum of each row
    # 
    
    # Apply sum-function to each row of dataframe and return the result
    rowsum <- apply(dataframe,1,sum)
    return(rowsum)
}
rowsum_apply <- rowSums_apply(data)

# Check that results match
all.equal(rowsum_forloop_cols,rowsum_apply)

[1] TRUE

Of course a language like R has an internal function for calculating the sum of columns for each row. This function is `rowSums`.

In [52]:
rowsum_baser <- rowSums(data)

# Check that results match
all.equal(rowsum_apply, rowsum_baser)


[1] TRUE

Now that we have five implementations of the same function we can use `microbenchmark`-library to make them compete against each other. `microbenchmark`-function can be used to run a function call in order to generate statistics of the function runtime. By default it runs the code 100 times.

In [56]:
library(microbenchmark)

print(microbenchmark(
    rowSums_forloop_rows(data),
    rowSums_forloop_cols(data),
    rowSums_forloop_colnames(data),
    rowSums_apply(data),
    rowSums(data)
))

Unit: microseconds
                           expr      min       lq       mean   median        uq
     rowSums_forloop_rows(data) 7175.152 7402.619 8102.11593 7674.836 8195.6115
     rowSums_forloop_cols(data) 6618.140 6835.628 7191.17820 6997.892 7468.7940
 rowSums_forloop_colnames(data)   56.499   62.267   67.84726   67.132   70.5875
            rowSums_apply(data)  261.372  284.286  318.63438  304.308  324.8640
                  rowSums(data)   52.931   59.906   84.29527   82.943   91.2400
       max neval
 37133.877   100
  8920.005   100
   165.555   100
  1027.494   100
   754.100   100


As is apparent from the runtimes the bad for-loop implementations were the ones that went through the data one element at a time while the one that added one column at a time to the sum was even better than the `apply`-function.

As a conclusion: The main reason to use `apply`-functions is *convenience* and efficiency through *minimizing risks*. With `apply`-functions you save code as you don't need to keep check of indices.

However do remember that `apply`-functions are meant to be used in cases where you can split input into independent chunks.

## Other *apply-functions

### lapply

`lapply` is similar to `apply` but it always operates on columns and it returns a list as its output. Call for this function is

`lapply(X, FUN)`

### sapply

`sapply`  is similar to `apply` but it always operates on columns and always simplifies to result to the most simple data type available. Call for this function is

`lapply(X, FUN)`

### vapply

`vapply`  is similar to `apply` but it always operates on columns and verifies that each call of `FUN` has the type and size of a vector given in `FUN.VALUE`. Call for this function is

`lapply(X, FUN, FUN.VALUE)`

### mapply

`mapply` or multivariate apply can be used to take one argument from one array, second from second, etc. and call a function `FUN` with these arguments. Call for this function is

`mapply(FUN, ...)`

where `...` are arrays given as positional arguments.

In [58]:
# Calculate mean of each column

apply_result <- apply(data,2,mean)
print('apply:')
str(apply_result)

lapply_result <- lapply(data,mean)
print('lapply:')
str(lapply_result)

sapply_result <- sapply(data,mean)
print('sapply:')
str(sapply_result)

vapply_result <- vapply(data,mean,complex(1))
print('vapply:')
str(vapply_result)

# Calculate sum for each row like in the previous example

mapply_result <- mapply(sum,data$var1,data$var2,data$var3,data$var4)
print('mapply:')
str(mapply_result)
all.equal(mapply_result,rowsum_baser)

[1] "apply:"
 Named num [1:4] 0.2388 -0.1031 0.0208 -0.1486
 - attr(*, "names")= chr [1:4] "var1" "var2" "var3" "var4"
[1] "lapply:"
List of 4
 $ var1: num 0.239
 $ var2: num -0.103
 $ var3: num 0.0208
 $ var4: num -0.149
[1] "sapply:"
 Named num [1:4] 0.2388 -0.1031 0.0208 -0.1486
 - attr(*, "names")= chr [1:4] "var1" "var2" "var3" "var4"
[1] "vapply:"
 Named cplx [1:4] 0.2388+0i -0.1031+0i 0.0208+0i ...
 - attr(*, "names")= chr [1:4] "var1" "var2" "var3" "var4"
[1] "mapply:"
 num [1:100] 5.298 -0.338 3.374 -0.985 -0.947 ...


[1] TRUE

## Maps